In [1]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOSuperMarioBrosZeroShot-v2" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 3 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

from helpers.utils import ENVIRONMENTS_MAP, get_algorithms
ALGORITHMS = get_algorithms(ENV_NAME)

### Normalize Front and Calculate Normalized Hypervolume and EUM for Generalist and Specialist

Import helpers

In [2]:
import numpy as np

sys.path.append(os.path.abspath(os.path.join('../..')))

def get_normalized_vec_returns(all_vec_returns, minmax_range):
    minmax_array = np.array([minmax_range[i] for i in range(all_vec_returns.shape[-1])])
    min_vals = minmax_array[:, 0].reshape(1, 1, -1) # reshape to (1, 1, n_objectives) for broadcasting
    max_vals = minmax_array[:, 1].reshape(1, 1, -1)

    clipped_vec_returns = np.clip(all_vec_returns, min_vals, max_vals) # broadcasted clipping
    
    # Normalize
    normalized_vec_returns = (clipped_vec_returns - min_vals) / (max_vals - min_vals)
    
    return normalized_vec_returns

In [3]:
from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100 # CHANGE THIS TO THE NUMBER OF WEIGHTS YOU WANT TO USE, NORMALLY ITS 100, FOR MARIO ITS 32
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 0., 1.]),
 array([0.        , 0.08214483, 0.91785517]),
 array([0.        , 0.16223292, 0.83776708]),
 array([0.       , 0.2406957, 0.7593043]),
 array([0.        , 0.31771743, 0.68228257])]

### Combine the fronts of all the Specialists for each environment

In [4]:
import warnings
from mo_utils.pareto import filter_pareto_dominated

curr_envs = ENVIRONMENTS_MAP[ENV_NAME]
SPECIALIST_FRONT = "eval/front" # don't change this, this is the discounted fronts but poorly named!!
path_to_find_fronts = f"data/single_env/{SPECIALIST_FRONT}/{ENV_NAME}"

for env in curr_envs:
    unfiltered_combined_front_df = None
    path_to_find_front_for_subenv = path_to_find_fronts + f"/{env}"
    
    for algo in ALGORITHMS:
        if os.path.exists(path_to_find_front_for_subenv + f"/{algo}.csv"):
            front_df = pd.read_csv(path_to_find_front_for_subenv + f"/{algo}.csv")
            print(f"Found front for {env} - {algo}, total row: {len(front_df)}")
            if unfiltered_combined_front_df is None:
                unfiltered_combined_front_df = front_df
            else:
                unfiltered_combined_front_df = pd.concat([unfiltered_combined_front_df, front_df])

    if unfiltered_combined_front_df is None:
        warnings.warn(f"No fronts found for {env}")
        continue
    
    unfiltered_combined_front_df = unfiltered_combined_front_df.reset_index(drop=True)
    print(f"Combined front for {env} has {len(unfiltered_combined_front_df)} rows")

    for column in unfiltered_combined_front_df.columns:
        min_value = unfiltered_combined_front_df[column].min()
        max_value = unfiltered_combined_front_df[column].max()
        print(f"{column}, Min: {min_value}, Max: {max_value}")

    combined_front_array = unfiltered_combined_front_df.to_numpy()
    filtered_combined_front_array = filter_pareto_dominated(combined_front_array)

    combined_front_df = pd.DataFrame(filtered_combined_front_array, columns=unfiltered_combined_front_df.columns)
    print(f"Filtered front for {env} has {len(combined_front_df)} rows")
    save_dir = f"data/single_env/combined_fronts/{ENV_NAME}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    combined_front_df.to_csv(f"{save_dir}/{env}.csv", index=False)

Found front for MOSuperMarioBros-2-3-v2 - GPI-LS, total row: 5
Combined front for MOSuperMarioBros-2-3-v2 has 5 rows
objective_1, Min: 14.895938292145727, Max: 30.70042108744383
objective_2, Min: -0.2496916595846414, Max: 26.67061138758436
objective_3, Min: -0.5403381395153701, Max: 7.234124569222331
Filtered front for MOSuperMarioBros-2-3-v2 has 5 rows
Found front for MOSuperMarioBros-3-2-v2 - GPI-LS, total row: 9
Combined front for MOSuperMarioBros-3-2-v2 has 9 rows
objective_1, Min: 20.316814068704844, Max: 29.48470828018617
objective_2, Min: 0.8370645992690697, Max: 3.726304597221315
objective_3, Min: 16.473927622660995, Max: 102.41305612551514
Filtered front for MOSuperMarioBros-3-2-v2 has 9 rows
Found front for MOSuperMarioBros-3-3-v2 - GPI-LS, total row: 3
Combined front for MOSuperMarioBros-3-3-v2 has 3 rows
objective_1, Min: 27.72344933144632, Max: 29.117828100919724
objective_2, Min: 16.888624942366732, Max: 22.312103191856295
objective_3, Min: 5.529992267023772, Max: 20.5473

### Normalize the fronts and calculate normalized hypervolume and EUM for SPECIALIST

In [23]:
from evaluation import get_eval_params

# These should exist in 'experiments/evaluation/eval_params.yaml' for current environment
# For new environments, you can add them to the file using the minmax values from the previous step
normalization_data = get_eval_params(ENV_NAME)['normalization']
normalization_data

{'MOLavaGridCheckerBoard-v0': {0: [0, 107.34], 1: [0, 218.76]},
 'MOLavaGridSmiley-v0': {0: [0, 270.69], 1: [0, 225.5]},
 'MOLavaGridSnake-v0': {0: [0, 234.21], 1: [0, 220.54]},
 'MOLavaGridIslands-v0': {0: [0, 124.23], 1: [0, 204.7]},
 'MOLavaGridLabyrinth-v0': {0: [0, 250.05], 1: [0, 226.95]},
 'MOLavaGridMaze-v0': {0: [0, 237.33], 1: [0, 203.59]},
 'MOLavaGridCorridor-v0': {0: [0, 265.66], 1: [0, 240.21]},
 'MOLavaGridRoom-v0': {0: [0, 263.86], 1: [0, 215.75]}}

In [24]:
from helpers.utils import ENVIRONMENTS_MAP

FRONT = "eval/discounted_front" # don't change this, front extracted for specialists are only the discounted ones!!
file_path = f"data/{FRONT}/{ENV_NAME}"
scores_save_path = f"data/scores/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

In [25]:
normalized_specialist_hypervolumes = []
normalized_specialist_eums = []

for env in ENVIRONMENTS_MAP[ENV_NAME]:
    min_max_ranges = normalization_data[env]
    best_env_front_path = f"data/single_env/combined_fronts/{ENV_NAME}/{env}.csv"
    assert os.path.exists(best_env_front_path), f"File {best_env_front_path} does not exist"
    
    best_env_front = pd.read_csv(best_env_front_path)
    data_array = best_env_front.to_numpy()
    normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)
    normalized_specialist_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
    normalized_specialist_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

specialist_data = {f"normalized_hypervolume/{env}": [normalized_specialist_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
specialist_data.update({f"normalized_eum/{env}": [normalized_specialist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
specialist_normalized_hv = pd.DataFrame(specialist_data)
specialist_normalized_hv.to_csv(f"{scores_save_path}/specialist.csv", index=False)

### Normalize the fronts and calculate normalized hypervolume and EUM for GENERALIST

In [26]:
# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        normalized_hypervolumes = []
        normalized_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            min_max_ranges = normalization_data[env]
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)

            normalized_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
            normalized_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

        data = {f"normalized_hypervolume/{env}": [normalized_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        data.update({f"normalized_eum/{env}": [normalized_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)
            

# Calculate NHGR

In [27]:
# get the normalized hypervolumes of the specialists
specialist_normalized_hv_data = pd.read_csv(f"{scores_save_path}/specialist.csv")

for algo in ALGORITHMS:
    for seed in SEEDS:
        nhgrs = []
        # get the normalized hypervolumes we extracted earlier
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_normalized_hv_data = pd.read_csv(file)

        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            # Filter columns that start with "normalized_hypervolume"
            col = f"normalized_hypervolume/{env}"

            specialist_normalized_hv = specialist_normalized_hv_data[col].values[0]
            generalist_normalized_hv = seed_normalized_hv_data[col].values[0]
            
            env_nhgr = generalist_normalized_hv / specialist_normalized_hv

            seed_normalized_hv_data[f'NHGR/{env}'] = env_nhgr
        
        seed_normalized_hv_data.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)